Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Inference PyTorch Bert Model for High Performance in ONNX Runtime

In this tutorial, you'll learn how to load a BERT model from PyTorch, convert it to ONNX, and inference it for high performance using ONNX Runtime with transformer optimization. In the following sections, we are going to use the BERT model trained with Stanford Question Answering Dataset (SQuAD) dataset as an example. BERT SQuAD model is used in question answering scenarios, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

### Tutorial Roadmap

During this tutorial, we will:
0. Install a few prerequisite packages (PyTorch, Transformers, TorchVision, wget).
1. Load a pretrained BERT SQuAD model from a source PyTorch implementation.
2. Export our PyTorch BERT model to ONNX.
3. Compare inference on our models for PyTorch and ONNX Runtime.
4. Optimize our inference with execution providers using the ONNX Go Live (OLive) tool.

## 0. Prerequisites ##
First you need to check if the following packages exist and install them if needed. 


In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install wget
!{sys.executable} -m pip install --user torch==1.3.1 torchvision==0.4.2+cpu -f https://download.pytorch.org/whl/torch_stable.html
!{sys.executable} -m pip install transformers==2.5.1
!{sys.executable} -m pip install psutil

Looking in links: https://download.pytorch.org/whl/torch_stable.html


## 1. Load Pretrained BERT model ##

We begin by downloading the data files and store them in the specified ``pytorch_output`` and ``pytorch_squad`` directories. 

In [2]:
import os

# Create a directory to store predict file
output_dir = "./pytorch_output"
cache_dir = "./pytorch_squad"
predict_file = os.path.join(cache_dir, "dev-v1.1.json")
# create cache dir
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
    
# Download the file
predict_file_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
if not os.path.exists(predict_file):
    import wget
    print("Start downloading predict file.")
    wget.download(predict_file_url, predict_file)
    print("Predict file downloaded.")

We specify some relevant model config / hyperparameter variables.

In [3]:
# Define some variables. As an example, we used batch size 1 and max sequence length 128. 
model_type = "bert"
model_name_or_path = "bert-base-cased"
max_seq_length = 128
doc_stride = 128
max_query_length = 64
eval_batch_size = 1
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   # The hardware you'd like to use to run the model.

Let's start to load our BERT model into PyTorch from our pretrained files. This step could take a few minutes. 

In [4]:
# The following code is adapted from HuggingFace transformers
# https://github.com/huggingface/transformers/blob/master/examples/run_squad.py#L290

from transformers import (WEIGHTS_NAME, BertConfig, BertForQuestionAnswering, BertTokenizer)
from torch.utils.data import (DataLoader, SequentialSampler)

# Load pretrained model and tokenizer
config_class, model_class, tokenizer_class = (BertConfig, BertForQuestionAnswering, BertTokenizer)
config = config_class.from_pretrained(model_name_or_path, cache_dir=cache_dir)
tokenizer = tokenizer_class.from_pretrained(model_name_or_path, do_lower_case=True, cache_dir=cache_dir)
model = model_class.from_pretrained(model_name_or_path,
                                    from_tf=False,
                                    config=config,
                                    cache_dir=cache_dir)

# Load and Convert the examples from the downloaded predict file into a list of features 
# that can be directly given as input to a model.
from transformers.data.processors.squad import SquadV2Processor

processor = SquadV2Processor()
examples = processor.get_dev_examples(None, filename=predict_file)

from transformers import squad_convert_examples_to_features
features, dataset = squad_convert_examples_to_features( 
            examples=examples[:3], # convert only 3 examples for demo
            tokenizer=tokenizer,
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_training=False,
            return_dataset='pt'
        )

c:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

### BERT Problem Formulation

**Input**: Context paragraph and questions

**Task**: For each question about the context paragraph, the model predicts a start and an end token from the paragraph that most likely answers the questions.

**Output**: The best answer for each question.

In [5]:
example_index = 200
example = examples[example_index]
print('qas_id: ', example.qas_id, '\n')
print('context_text: ', example.context_text, '\n')
print('question_text: ', example.question_text, '\n')
print('answer_text: ', example.answer_text, '\n')
print('title: ', example.title, '\n')
print('is_impossible: ', example.is_impossible, '\n')
print('answers: ', example.answers)

qas_id:  56beb3083aeaaa14008c923e 

context_text:  Despite waiving longtime running back DeAngelo Williams and losing top wide receiver Kelvin Benjamin to a torn ACL in the preseason, the Carolina Panthers had their best regular season in franchise history, becoming the seventh team to win at least 15 regular season games since the league expanded to a 16-game schedule in 1978. Carolina started the season 14–0, not only setting franchise records for the best start and the longest single-season winning streak, but also posting the best start to a season by an NFC team in NFL history, breaking the 13–0 record previously shared with the 2009 New Orleans Saints and the 2011 Green Bay Packers. With their NFC-best 15–1 regular season record, the Panthers clinched home-field advantage throughout the NFC playoffs for the first time in franchise history. Ten players were selected to the Pro Bowl (the most in franchise history) along with eight All-Pro selections. 

question_text:  Who had the b

## 2. Export the loaded model ##
Once the model is loaded, we can export the loaded PyTorch model to ONNX.

In [6]:
# Eval!
print("***** Running evaluation {} *****")
print("  Num examples = ", len(dataset))
print("  Batch size = ", eval_batch_size)

# create output dir
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
output_model_path = './pytorch_squad/bert-base-cased-squad.onnx'    
inputs = {}
outputs= {}
# Get the first batch of data to run the model and export it to ONNX
batch = dataset[0]

# Set model to inference mode, which is required before exporting the model because some operators behave differently in 
# inference and training mode.
model.eval()
batch = tuple(t.to(device) for t in batch)
inputs = {
    'input_ids':      batch[0].reshape(1, 128),                         # using batch size = 1 here. Adjust as needed.
    'attention_mask': batch[1].reshape(1, 128),
    'token_type_ids': batch[2].reshape(1, 128)
}

with torch.no_grad():
    symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}
    torch.onnx.export(model,                                            # model being run
                      (inputs['input_ids'],                             # model input (or a tuple for multiple inputs)
                       inputs['attention_mask'], 
                       inputs['token_type_ids']), 
                      output_model_path,                                # where to save the model (can be a file or file-like object)
                      opset_version=11,                                 # the ONNX version to export the model to
                      do_constant_folding=True,                         # whether to execute constant folding for optimization
                      input_names=['input_ids',                         # the model's input names
                                   'input_mask', 
                                   'segment_ids'],
                      output_names=['start', 'end'],                    # the model's output names
                      dynamic_axes={'input_ids': symbolic_names,        # variable length axes
                                    'input_mask' : symbolic_names,
                                    'segment_ids' : symbolic_names,
                                    'start' : symbolic_names,
                                    'end' : symbolic_names})
    print("Model exported at ", output_model_path)

***** Running evaluation {} *****
  Num examples =  6
  Batch size =  1


C:\Users\viswamy\AppData\Roaming\Python\Python36\site-packages\torch\onnx\symbolic_opset9.py:427: UserWarning: ONNX export squeeze with negative axis -1 might cause the onnx model to be incorrect. Negative axis is not supported in ONNX. Axis is converted to 2 based on input shape at export time. Passing an tensor of different rank in execution will be incorrect.
  "Passing an tensor of different rank in execution will be incorrect.")


Model exported at  ./pytorch_squad/bert-base-cased-squad.onnx


## 3. Inference the Exported Model with ONNX Runtime ##

#### Install ONNX Runtime
Install ONNX Runtime if you haven't done so already. Make sure to install the correct package from PyPi -- `onnxruntime` to use CPU features, or `onnxruntime-gpu` to use GPU. 

In [7]:
ONNXRUNTIME = 'onnxruntime'
# Install ONNX Runtime
if torch.cuda.is_available():
    ## Install onnxruntime-gpu if cuda is available
    ONNXRUNTIME = 'onnxruntime-gpu'

import sys
!{sys.executable} -m pip install -U $ONNXRUNTIME

Requirement already up-to-date: onnxruntime in c:\anaconda3\lib\site-packages (1.2.0)


Now we are ready to inference our model with ONNX Runtime!

In [8]:
import onnxruntime as rt  
import time
import psutil

sess_options = rt.SessionOptions()

# Set graph optimization level to ORT_ENABLE_EXTENDED to enable bert optimization. This is enabled on default.
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_EXTENDED

# The following settings enables OpenMP, which is required to get best performance for CPU inference of Bert models.
sess_options.intra_op_num_threads=1
os.environ["OMP_NUM_THREADS"] = str(psutil.cpu_count(logical=True))
os.environ["OMP_WAIT_POLICY"] = 'ACTIVE'

session = rt.InferenceSession(output_model_path, sess_options)

# evaluate the model
start = time.time()
res = session.run(None, {
          'input_ids': inputs['input_ids'].cpu().numpy(),
          'input_mask': inputs['attention_mask'].cpu().numpy(),
          'segment_ids': inputs['token_type_ids'].cpu().numpy()
        })
end = time.time()
print("ONNX Runtime inference time: ", end - start)

ONNX Runtime inference time:  0.13757848739624023


Get comparative performance numbers from the original PyTorch model.

In [9]:
start = time.time()
outputs = model(**inputs)
end = time.time()
print("PyTorch Inference time = ", end - start)

print("***** Verifying correctness *****")
import numpy as np
for i in range(2):
    print('PyTorch and ORT matching numbers:', np.allclose(res[i], outputs[i].cpu().detach().numpy(), rtol=1e-04, atol=1e-05))


PyTorch Inference time =  0.28899693489074707
***** Verifying correctness *****
PyTorch and ORT matching numbers: True
PyTorch and ORT matching numbers: True


## 4. Tune Best Performance with OLive tool

To get a better performance, we can tune parameters and try different execution providers in ONNX Runtime. The [OLive(ONNX Go Live) tool](https://github.com/microsoft/OLive) provides docker images as well as a web app to help finding the best parameters and execution provider for a specific model. 

![OLive-Demo](img/OLive_demo_pic.png)